In [12]:
%load_ext nb_black

The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black


<IPython.core.display.Javascript object>

In [13]:
import os
import re
import time

import pandas as pd
import requests
import selenium
from bs4 import BeautifulSoup
from selenium import webdriver
from tqdm.notebook import tqdm

<IPython.core.display.Javascript object>

In [14]:
def browser(url):
    option = webdriver.ChromeOptions()
    option.add_argument('headless')
    driver = webdriver.Chrome(options=option)
    driver.get(url)
    return driver


def get_html_full(url):
    driver = browser(url)
    el = driver.find_element_by_xpath(
        "/html/body/of-root/main/of-competition-results-stream/section/of-entity-page-root/div/of-xpa-switch-entity-deprecated[7]/of-simple-match-cards-list-deprecated/div/button"
    )
    el.click()
    time.sleep(5)
    page = driver.page_source
    driver.close()
    return BeautifulSoup(page)


def get_html_full_match(url):
    driver = browser(url)
    el = driver.find_element_by_xpath(
        "/html/body/of-root/main/of-competition-results-stream/section/of-entity-page-root/div/of-xpa-switch-entity-deprecated[7]/of-simple-match-cards-list-deprecated/div/button"
    )
    el.click()
    time.sleep(5)
    page = driver.page_source
    driver.close()
    return BeautifulSoup(page)


def get_html_from_a_match(url):
    driver = browser(url)
    el = driver.find_element_by_xpath(
        "/html/body/of-root/main/of-match-stream-v2/section/of-xpa-layout-match/section[2]/of-xpa-switch-match/of-match-events/div/button"
    )
    el.click()
    time.sleep(2)
    page = driver.page_source
    el = driver.find_element_by_xpath(
        "/html/body/of-root/main/of-match-stream-v2/section/of-xpa-layout-match/section[3]/div[1]/div/of-xpa-switch-match[2]/of-match-lineup/div/nav/ul/li[2]/button"
    )
    el.click()
    time.sleep(2)
    second_lineup = driver.page_source
    driver.close()
    return BeautifulSoup(page), BeautifulSoup(second_lineup)



def format_field(string):
    string = str(string)
    string = string.lower()
    string = string.strip()
    string = string.replace("  ", " ")
    string = string.replace(" ", "_")
    return string


def get_info_from_event(event):
    aux_event = {}
    aux_event["event_team"] = event["class"][-1][-4:]
    aux_event["event_time"] = event.find(
        "div", class_="match-events__item-timeline"
    ).text

    if aux_event["event_time"] == " PK ":
        aux_event["event_time"] = False
        aux_event["event_type"] = "PK"
        aux_event["event_result"] = event.find("img", class_="of-image__img")["alt"]
        aux_event["event_player"] = (
            event.find("div", class_="match-events__text").find("p").text
        )
    else:
        try:
            aux_event["event_type"] = event.find(class_="match-events__icon")[
                "aria-label"
            ]
        except:
            aux_event["event_type"] = event.find("img", class_="of-image__img")["alt"]
        try:
            for i, text in enumerate(
                event.find("div", class_="match-events__text").find_all("p")
            ):
                aux_event["action_player_" + str(i + 1)] = text.text
        except:
            aux_event["action_player_1"] = event.find(
                "p", class_="match-events__text"
            ).text

    return aux_event


def get_info_from_match(page, second_lineup):
    aux_dict = {}

    ## NAMES
    aux_dict["team_name_home"] = page.find(
        "span", class_="match-score-team__name--home"
    ).text
    aux_dict["team_name_away"] = page.find(
        "span", class_="match-score-team__name--away"
    ).text

    ## GOAL
    scores = page.find("p", class_="match-score-scores").find_all("span")
    aux_dict["team_home_score"] = scores[0].text
    aux_dict["team_away_score"] = scores[2].text

    ## STATISTICS
    description = page.find_all("div", class_="match-stats__stat-description")
    for d in description:
        field = format_field(d.find_all("p")[1].text)
        aux_dict[field + "_home"] = d.find_all("p")[0].text
        aux_dict[field + "_away"] = d.find_all("p")[2].text

    ## EVENTS
    events = page.find("ul", class_="match-events__list").find_all(
        "li", class_="match-events__item"
    )
    aux_dict["events_list"] = [get_info_from_event(event) for event in events]

    aux_dict["lineup_home"] = get_lineups(page)
    aux_dict["lineup_away"] = get_lineups(second_lineup)

    return aux_dict


def get_lineups(page):
    lineup = []
    for player in page.find_all("div", class_="match-lineup__player"):
        lineup.append(
            {
                "Player_Name": player.find(
                    "p", class_="match-lineup__player-name"
                ).text,
                "Player_Number": format_field(
                    player.find("span", class_="match-lineup__jersey-number").text
                ),
            }
        )
    return lineup



def get_penaltis(match):
    regex_rule = r"\((\d+)\)"
    aux = match.find_all(
        "span", class_="title-7-bold simple-match-card-team__score"
    )
    aux = [
        re.search(regex_rule, i.text)[0].replace("(", "").replace(")", "") for i in aux
    ]
    return aux


def get_all_matches(html_full):
    aux_dict = []
    for match in html_full.find_all("a", class_="match-card", href=True):
        aux_dict_2 = {}
        aux_dict_2["link"] = "https://onefootball.com" + match["href"]
        aux_dict_2["stage"] = match.find_previous(
            "p", class_="label simple-match-cards-list__title-deprecated"
        ).text
        try:
            aux_dict_2["date"] = match.find("time").text
            aux_dict_2["pens"] = False
            aux_dict_2["pens_home_score"] = False
            aux_dict_2["pens_away_score"] = False
        except:
            aux_dict_2["date"] = False
            aux_dict_2["pens"] = (
                match.find(
                    "span",
                    class_="title-8-medium simple-match-card__info-message simple-match-card__info-message--secondary",
                ).text
                == "(Pens)"
            )
            home, away = get_penaltis(match)
            aux_dict_2["pens_home_score"] = home
            aux_dict_2["pens_away_score"] = away

        aux_dict.append(aux_dict_2)
    return aux_dict


<IPython.core.display.Javascript object>

## Competition

In [15]:
url = "https://onefootball.com/en/competition/copa-america-37/results"  ## Competition link
page = get_html_full(url)
matchs = get_all_matches(page)
matchs[0]

{'link': 'https://onefootball.com/en/match/2204580',
 'stage': ' Final ',
 'date': ' 10.07.2021 ',
 'pens': False,
 'pens_home_score': False,
 'pens_away_score': False}

<IPython.core.display.Javascript object>

In [16]:
%%time

def merge_information(match):
    match_info = get_info_from_match(*get_html_from_a_match(match["link"]))
    return{**match, **match_info}
    
from joblib import Parallel, delayed
aux_list = Parallel(n_jobs=6, verbose=10)(delayed(merge_information)(match) for match in matchs)

[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   1 tasks      | elapsed:   10.1s
[Parallel(n_jobs=6)]: Done   6 tasks      | elapsed:   10.5s
[Parallel(n_jobs=6)]: Done  13 tasks      | elapsed:   29.7s
[Parallel(n_jobs=6)]: Done  20 out of  28 | elapsed:   43.7s remaining:   17.4s
[Parallel(n_jobs=6)]: Done  23 out of  28 | elapsed:   54.2s remaining:   11.7s
[Parallel(n_jobs=6)]: Done  26 out of  28 | elapsed:  1.0min remaining:    4.7s


Wall time: 1min 4s


[Parallel(n_jobs=6)]: Done  28 out of  28 | elapsed:  1.1min finished


<IPython.core.display.Javascript object>

In [17]:
df = pd.DataFrame(aux_list).drop("link", axis=1)
df.shape

(28, 20)

<IPython.core.display.Javascript object>

## Manual adjustment

In [18]:
df[df.date == False]

,stage,date,pens,pens_home_score,pens_away_score,team_name_home,team_name_away,team_home_score,team_away_score,possession_home,possession_away,total_shots_home,total_shots_away,shots_on_target_home,shots_on_target_away,duels_won_home,duels_won_away,events_list,lineup_home,lineup_away
2,Semi-finals,False,True,3,2,Argentina,Colombia,1,1,49%,51%,13,14,4,4,53%,47%,"[{'event_team': 'home', 'event_time': ' 7' ', ...","[{'Player_Name': 'González', 'Player_Number': ...","[{'Player_Name': 'Zapata', 'Player_Number': '7..."
5,Quarter-finals,False,True,2,4,Uruguay,Colombia,0,0,49%,51%,7,9,3,3,53%,47%,"[{'event_team': 'home', 'event_time': ' 35' ',...","[{'Player_Name': 'Cavani', 'Player_Number': '2...","[{'Player_Name': 'Zapata', 'Player_Number': '7..."
7,Quarter-finals,False,True,4,3,Peru,Paraguay,3,3,55%,45%,12,14,5,8,44%,56%,"[{'event_team': 'away', 'event_time': ' 11' ',...","[{'Player_Name': 'Lapadula Vargas', 'Player_Nu...","[{'Player_Name': 'Sánchez Guerrero', 'Player_N..."


<IPython.core.display.Javascript object>

In [19]:
df.date.iloc[2] = "06.07.2021"  ## Argentina x Colombia
df.date.iloc[5] = "03.07.2021"  ## Uruguay x Colombia
df.date.iloc[7] = "02.07.2021"  ## Peru x Paraguay

C:\Users\marco_carujo\AppData\Roaming\Python\Python38\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


<IPython.core.display.Javascript object>

In [20]:
df.to_csv("copa_america_2021_results.csv", index=False)

<IPython.core.display.Javascript object>

In [21]:
df

,stage,date,pens,pens_home_score,pens_away_score,team_name_home,team_name_away,team_home_score,team_away_score,possession_home,possession_away,total_shots_home,total_shots_away,shots_on_target_home,shots_on_target_away,duels_won_home,duels_won_away,events_list,lineup_home,lineup_away
0,Final,10.07.2021,False,False,False,Argentina,Brazil,1,0,40%,60%,6,13,2,2,47%,53%,"[{'event_team': 'away', 'event_time': ' 3' ', ...","[{'Player_Name': 'Martínez', 'Player_Number': ...","[{'Player_Name': 'Richarlison', 'Player_Number..."
1,Match for third place,09.07.2021,False,False,False,Colombia,Peru,3,2,45%,55%,12,10,6,3,55%,46%,"[{'event_team': 'away', 'event_time': ' 24' ',...","[{'Player_Name': 'Zapata', 'Player_Number': '7...","[{'Player_Name': 'Lapadula Vargas', 'Player_Nu..."
2,Semi-finals,06.07.2021,True,3,2,Argentina,Colombia,1,1,49%,51%,13,14,4,4,53%,47%,"[{'event_team': 'home', 'event_time': ' 7' ', ...","[{'Player_Name': 'González', 'Player_Number': ...","[{'Player_Name': 'Zapata', 'Player_Number': '7..."
3,Semi-finals,05.07.2021,False,False,False,Brazil,Peru,1,0,56%,44%,15,7,8,2,55%,45%,"[{'event_team': 'home', 'event_time': ' 35' ',...","[{'Player_Name': 'Richarlison', 'Player_Number...","[{'Player_Name': 'Lapadula Vargas', 'Player_Nu..."
4,Quarter-finals,03.07.2021,False,False,False,Argentina,Ecuador,3,0,46%,54%,21,10,8,2,52%,48%,"[{'event_team': 'away', 'event_time': ' 20' ',...","[{'Player_Name': 'González', 'Player_Number': ...","[{'Player_Name': 'Palacios Espinoza', 'Player_..."
5,Quarter-finals,03.07.2021,True,2,4,Uruguay,Colombia,0,0,49%,51%,7,9,3,3,53%,47%,"[{'event_team': 'home', 'event_time': ' 35' ',...","[{'Player_Name': 'Cavani', 'Player_Number': '2...","[{'Player_Name': 'Zapata', 'Player_Number': '7..."
6,Quarter-finals,02.07.2021,False,False,False,Brazil,Chile,1,0,41%,59%,10,11,5,5,50%,50%,"[{'event_team': 'home', 'event_time': ' 45' ',...","[{'Player_Name': 'Neymar', 'Player_Number': '1...","[{'Player_Name': 'Sanchez', 'Player_Number': '..."
7,Quarter-finals,02.07.2021,True,4,3,Peru,Paraguay,3,3,55%,45%,12,14,5,8,44%,56%,"[{'event_team': 'away', 'event_time': ' 11' ',...","[{'Player_Name': 'Lapadula Vargas', 'Player_Nu...","[{'Player_Name': 'Sánchez Guerrero', 'Player_N..."
8,Group stage: Matchday 5,28.06.2021,False,False,False,Uruguay,Paraguay,1,0,48%,52%,18,4,6,0,57%,43%,"[{'event_team': 'home', 'event_time': ' 21' ',...","[{'Player_Name': 'Cavani', 'Player_Number': '2...","[{'Player_Name': 'Ávalos Stumpfs', 'Player_Num..."
9,Group stage: Matchday 5,28.06.2021,False,False,False,Bolivia,Argentina,1,4,32%,68%,5,18,2,13,42%,58%,"[{'event_team': 'away', 'event_time': ' 6' ', ...","[{'Player_Name': 'Chura Almanza', 'Player_Numb...","[{'Player_Name': 'Agüero', 'Player_Number': '9..."


<IPython.core.display.Javascript object>